In [4]:
import requests
import pandas as pd
from pathlib import Path


import requests, json, time
import pandas as pd
from pathlib import Path


ENTRY_ID = 1451223
SEASON   = 2024              # 2024-25
BASE     = "https://fantasy.premierleague.com/api"
OUTDIR   = Path("../data/external/2024-25")
OUTDIR.mkdir(parents=True, exist_ok=True)       # reuse TCP & cookies

In [5]:
def get(url, retry=5):
    for _ in range(retry):
        try:
            r = requests.get(url, timeout=10)
            if r.status_code == 200:
                return r.json()
        except requests.RequestException:
            pass
        time.sleep(3)
    raise RuntimeError(f"GET failed: {url}")

In [6]:
import requests, pandas as pd

data   = requests.get("https://fantasy.premierleague.com/api/bootstrap-static/").json()
prices = (
    pd.json_normalize(data["elements"])          # every registered player
      .loc[:, ["id", "web_name", "first_name", "second_name", "team", "element_type", "now_cost"]]
      .assign(price_m = lambda d: d.now_cost / 10)   # £m, human-readable
)

prices.to_csv(OUTDIR / "player_prices_2025-26.csv", index=False)
print(prices.head())


   id      web_name first_name            second_name  team  element_type  \
0   1          Raya      David            Raya Martín     1             1   
1   2  Arrizabalaga       Kepa  Arrizabalaga Revuelta     1             1   
2   3          Hein       Karl                   Hein     1             1   
3   4       Setford      Tommy                Setford     1             1   
4   5       Gabriel    Gabriel   dos Santos Magalhães     1             2   

   now_cost  price_m  
0        55      5.5  
1        45      4.5  
2        40      4.0  
3        40      4.0  
4        60      6.0  


In [7]:
hist = get(f"{BASE}/entry/{ENTRY_ID}/history/?season={SEASON}")
pd.DataFrame(hist["past"]).to_csv(OUTDIR/"history.csv", index=False)
pd.DataFrame(hist["current"]).to_csv(OUTDIR/"gw_summary.csv", index=False)